In [116]:
import pandas as pd
from utils import aggregate_data, join_col_descriptions, categorize_school_level, PostGresLoad

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [117]:
pg = PostGresLoad()

In [50]:
prev_year_data = pd.read_csv('/Users/cave/Desktop/pleiades_archive/discriminology/output/db_upload_schools.csv', index_col=0).rename(columns={'index': 'COMBOKEY'})
prev_year_data.head()

/Users/cave/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (1,3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,COMBOKEY,LEA_STATE_NAME,LEAID,LEA_NAME,SCHID,SCH_NAME,JJ,SCH_GRADE_PS,SCH_GRADE_KG,SCH_GRADE_G01,...,SCH_DISCWDIS_ARR_IDEA_HP_F,SCH_DISCWDIS_ARR_IDEA_TR_F,TOT_DISCWDIS_ARR_IDEA_F,YEAR,LEA_STATE,SCH_FTESECURITY_IND,SCH_ADDRESS,SCH_CITY,SCH_ZIP,SCH_LEVEL
0,10000201705,ALABAMA,100002,Alabama Youth Services,1705,Wallace Sch - Mt Meigs Campus,Yes,No,No,No,...,0.0,0.0,0.0,2015-16,AL,NaN,P O Box 66,Mount Meigs,36057,Secondary School
1,10000201705,ALABAMA,100002,Alabama Youth Services,1705,Wallace Sch - Mt Meigs Campus,Yes,No,No,No,...,0.0,0.0,0.0,2013-14,AL,No,P O Box 66,Mount Meigs,36057,Secondary School
2,10000201705,ALABAMA,100002,Alabama Youth Services,1705,Wallace Sch - Mt Meigs Campus,Yes,No,No,No,...,0.0,0.0,0.0,2011-12,AL,No,P O Box 66,Mount Meigs,36057,Secondary School
3,10000201706,ALABAMA,100002,Alabama Youth Services,1706,Mcneel Sch - Vacca Campus,Yes,No,No,No,...,0.0,0.0,0.0,2011-12,AL,No,8950 Roebuck Blvd,Birmingham,35206,Secondary School
4,10000201706,ALABAMA,100002,Alabama Youth Services,1706,Mcneel Sch - Vacca Campus,Yes,No,No,No,...,0.0,0.0,0.0,2013-14,AL,No,8950 Roebuck Blvd,Birmingham,35206,Secondary School


Read field mapping and data types from [dictionary](https://docs.google.com/spreadsheets/d/1mO7Tnbn9hwHcJ0hj8gBBp5MhHGDqmgGw8X3-b-s6SKs/edit#gid=0)

In [51]:
field_mappings = pd.read_excel('/Users/cave/Desktop/pleiades_archive/discriminology/field_mapping.xlsx')

In [52]:
field_mappings.head()

,Unnamed: 0,11_12_field,13_14_field,15_16_field,17_18_field,description,Include in Schools tab? (Limit 16),Include in Districts tab? (Limit 40),type,Unnamed: 9,...,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27
0,1,LEA_STATE,LEA_STATE,LEA_STATE_NAME,LEA_STATE_NAME,District State Name,yes,yes,str,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,LEAID,LEAID,LEAID,LEAID,7 Digit LEAID District Identification Code,yes,yes,str,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,LEA_NAME,LEA_NAME,LEA_NAME,LEA_NAME,District Name,yes,yes,str,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,SCHID,SCHID,SCHID,SCHID,5 Digit School Identification Code,no,no,str,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,SCH_NAME,SCH_NAME,SCH_NAME,SCH_NAME,School Name,yes,no,str,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
field_mappings['col_superset'] = field_mappings['15_16_field'].combine_first(field_mappings['11_12_field'])
type_map = pd.Series(field_mappings.type.values, index=field_mappings.col_superset).to_dict()
numeric_type_map = {k:v for k,v in type_map.items() if v !='str'}

In [54]:
newark = pd.read_csv('/Users/cave/Desktop/pleiades_archive/discriminology/newark_schools.csv', dtype={'COMBOKEY':str})
newark.head()

,SCH_NAME,COMBOKEY
0,ABINGTON AVENUE SCHOOL,341134002222
1,AMERICAN HISTORY HIGH SCHOOL,341134000791
2,ANN STREET SCHOOL,341134002228
3,ARTS HIGH SCHOOL,341134002188
4,AVON AVENUE SCHOOL,341134002232


### Extract methodology fields for each year

In [55]:
fields_1112 = field_mappings['11_12_field'].dropna().values
fields_1314 = field_mappings['13_14_field'].dropna().values
fields_1516 = field_mappings['15_16_field'].dropna().values
fields_1718 = field_mappings['17_18_field'].dropna().values

In [56]:
cd ~/Desktop/pleiades_archive/discriminology/2017_18/Data/SCH/CRDC/CSV

/Users/cave/Desktop/pleiades_archive/discriminology/2017_18/Data/SCH/CRDC/CSV


### Loop through files in 2017-18 OCR dump and pull out variables of interest.

In [13]:
df_17_18 = pd.read_csv('Enrollment.csv',  dtype={'COMBOKEY':str})
files_17_18 = ['Gifted and Talented.csv', 'Suspensions.csv', 'Advanced Placement.csv', 'School Characteristics.csv',
              'Referrals and Arrests.csv', 'Single-sex Classes.csv', 'School Support.csv'
              ]

join_keys = ['LEA_NAME', 'SCH_NAME', 'SCHID', 'LEAID', 'JJ', 'LEA_STATE_NAME']
for file in files_17_18:
    temp = pd.read_csv(file, dtype={'COMBOKEY':str})
    col_subset = [c for c in temp.columns if c not in join_keys]
    df_17_18 = pd.merge(df_17_18, temp[col_subset], on='COMBOKEY')


/Users/cave/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [57]:
df_17_18 = df_17_18[fields_1516]
df_17_18.head()

,LEA_STATE_NAME,LEAID,LEA_NAME,SCHID,SCH_NAME,JJ,SCH_GRADE_PS,SCH_GRADE_KG,SCH_GRADE_G01,SCH_GRADE_G02,...,SCH_DISCWDIS_ARR_IDEA_TR_M,TOT_DISCWDIS_ARR_IDEA_M,SCH_DISCWDIS_ARR_IDEA_AM_F,SCH_DISCWDIS_ARR_IDEA_AS_F,SCH_DISCWDIS_ARR_IDEA_HI_F,SCH_DISCWDIS_ARR_IDEA_BL_F,SCH_DISCWDIS_ARR_IDEA_WH_F,SCH_DISCWDIS_ARR_IDEA_HP_F,SCH_DISCWDIS_ARR_IDEA_TR_F,TOT_DISCWDIS_ARR_IDEA_F
0,ALABAMA,0100002,Alabama Youth Services,01705,Wallace Sch - Mt Meigs Campus,Yes,No,No,No,No,...,0,0,0,0,0,0,0,0,0,0
1,ALABAMA,0100002,Alabama Youth Services,01706,McNeel Sch - Vacca Campus,Yes,No,No,No,No,...,0,0,0,0,0,0,0,0,0,0
2,ALABAMA,0100002,Alabama Youth Services,01876,Alabama Youth Services,No,No,No,No,No,...,0,0,0,0,0,0,0,0,0,0
3,ALABAMA,0100002,Alabama Youth Services,99995,AUTAUGA CAMPUS,Yes,No,No,No,No,...,0,0,0,0,0,0,0,0,0,0
4,ALABAMA,0100005,Albertville City,00870,Albertville Middle School,No,No,No,No,No,...,0,0,0,0,0,0,0,0,0,0


In [58]:
df_17_18.shape

(97632, 226)

### Check how many Newark schools are missing from the 17-18 data

In [59]:
len(newark)
merged = pd.merge(newark, df_17_18, on='COMBOKEY', how='left')
merged['LEAID'].isna().sum()

4

Generate SQL create statement for school table in AWS

In [60]:
create_school_table = [print(f'"{k}" double precision,') for k,v in numeric_type_map.items()]
create_school_table

"SCH_FTECOUNSELORS" double precision,
"SCH_FTESERVICES_PSY" double precision,
"SCH_FTESERVICES_SOC" double precision,
"SCH_FTESECURITY_LEO" double precision,
"SCH_FTESECURITY_GUA" double precision,
"SCH_ENR_AM_M" double precision,
"SCH_ENR_AS_M" double precision,
"SCH_ENR_HI_M" double precision,
"SCH_ENR_BL_M" double precision,
"SCH_ENR_WH_M" double precision,
"SCH_ENR_HP_M" double precision,
"SCH_ENR_TR_M" double precision,
"TOT_ENR_M" double precision,
"SCH_ENR_AM_F" double precision,
"SCH_ENR_AS_F" double precision,
"SCH_ENR_HI_F" double precision,
"SCH_ENR_BL_F" double precision,
"SCH_ENR_WH_F" double precision,
"SCH_ENR_HP_F" double precision,
"SCH_ENR_TR_F" double precision,
"TOT_ENR_F" double precision,
"SCH_GTENR_AM_M" double precision,
"SCH_GTENR_AS_M" double precision,
"SCH_GTENR_HI_M" double precision,
"SCH_GTENR_BL_M" double precision,
"SCH_GTENR_WH_M" double precision,
"SCH_GTENR_HP_M" double precision,
"SCH_GTENR_TR_M" double precision,
"TOT_GTENR_M" double precision,
"SC

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [61]:
universal_types = {'SCH_ZIP ': str
                   ,'SCHID': str
                   ,'COMBOKEY': str
                   ,'LEAID': str
                  }

### Clean up data types

In [62]:
df_17_18['LEAID'] = df_17_18['LEAID'].astype(str).str.zfill(7)
df_17_18['SCHID'] = df_17_18['SCHID'].astype(str).str.zfill(5)
df_17_18['COMBOKEY'] = df_17_18['LEAID'] + df_17_18['SCHID']

In [63]:
df_17_18.head()

,LEA_STATE_NAME,LEAID,LEA_NAME,SCHID,SCH_NAME,JJ,SCH_GRADE_PS,SCH_GRADE_KG,SCH_GRADE_G01,SCH_GRADE_G02,...,SCH_DISCWDIS_ARR_IDEA_TR_M,TOT_DISCWDIS_ARR_IDEA_M,SCH_DISCWDIS_ARR_IDEA_AM_F,SCH_DISCWDIS_ARR_IDEA_AS_F,SCH_DISCWDIS_ARR_IDEA_HI_F,SCH_DISCWDIS_ARR_IDEA_BL_F,SCH_DISCWDIS_ARR_IDEA_WH_F,SCH_DISCWDIS_ARR_IDEA_HP_F,SCH_DISCWDIS_ARR_IDEA_TR_F,TOT_DISCWDIS_ARR_IDEA_F
0,ALABAMA,0100002,Alabama Youth Services,01705,Wallace Sch - Mt Meigs Campus,Yes,No,No,No,No,...,0,0,0,0,0,0,0,0,0,0
1,ALABAMA,0100002,Alabama Youth Services,01706,McNeel Sch - Vacca Campus,Yes,No,No,No,No,...,0,0,0,0,0,0,0,0,0,0
2,ALABAMA,0100002,Alabama Youth Services,01876,Alabama Youth Services,No,No,No,No,No,...,0,0,0,0,0,0,0,0,0,0
3,ALABAMA,0100002,Alabama Youth Services,99995,AUTAUGA CAMPUS,Yes,No,No,No,No,...,0,0,0,0,0,0,0,0,0,0
4,ALABAMA,0100005,Albertville City,00870,Albertville Middle School,No,No,No,No,No,...,0,0,0,0,0,0,0,0,0,0


### Replace negative values with zeroes.

In [64]:
num = df_17_18._get_numeric_data()
num[num < 0] = 0
df_17_18 = df_17_18.replace('-5', 'No').replace('-9', 'No')
df_1718_final = df_17_18
df_1718_final['YEAR'] = '2017-18'
df_1718_final.head()

,LEA_STATE_NAME,LEAID,LEA_NAME,SCHID,SCH_NAME,JJ,SCH_GRADE_PS,SCH_GRADE_KG,SCH_GRADE_G01,SCH_GRADE_G02,...,TOT_DISCWDIS_ARR_IDEA_M,SCH_DISCWDIS_ARR_IDEA_AM_F,SCH_DISCWDIS_ARR_IDEA_AS_F,SCH_DISCWDIS_ARR_IDEA_HI_F,SCH_DISCWDIS_ARR_IDEA_BL_F,SCH_DISCWDIS_ARR_IDEA_WH_F,SCH_DISCWDIS_ARR_IDEA_HP_F,SCH_DISCWDIS_ARR_IDEA_TR_F,TOT_DISCWDIS_ARR_IDEA_F,YEAR
0,ALABAMA,0100002,Alabama Youth Services,01705,Wallace Sch - Mt Meigs Campus,Yes,No,No,No,No,...,0,0,0,0,0,0,0,0,0,2017-18
1,ALABAMA,0100002,Alabama Youth Services,01706,McNeel Sch - Vacca Campus,Yes,No,No,No,No,...,0,0,0,0,0,0,0,0,0,2017-18
2,ALABAMA,0100002,Alabama Youth Services,01876,Alabama Youth Services,No,No,No,No,No,...,0,0,0,0,0,0,0,0,0,2017-18
3,ALABAMA,0100002,Alabama Youth Services,99995,AUTAUGA CAMPUS,Yes,No,No,No,No,...,0,0,0,0,0,0,0,0,0,2017-18
4,ALABAMA,0100005,Albertville City,00870,Albertville Middle School,No,No,No,No,No,...,0,0,0,0,0,0,0,0,0,2017-18


In [65]:
df_1718_final = df_1718_final.astype(numeric_type_map)
df_1718_final.head()

,LEA_STATE_NAME,LEAID,LEA_NAME,SCHID,SCH_NAME,JJ,SCH_GRADE_PS,SCH_GRADE_KG,SCH_GRADE_G01,SCH_GRADE_G02,...,TOT_DISCWDIS_ARR_IDEA_M,SCH_DISCWDIS_ARR_IDEA_AM_F,SCH_DISCWDIS_ARR_IDEA_AS_F,SCH_DISCWDIS_ARR_IDEA_HI_F,SCH_DISCWDIS_ARR_IDEA_BL_F,SCH_DISCWDIS_ARR_IDEA_WH_F,SCH_DISCWDIS_ARR_IDEA_HP_F,SCH_DISCWDIS_ARR_IDEA_TR_F,TOT_DISCWDIS_ARR_IDEA_F,YEAR
0,ALABAMA,0100002,Alabama Youth Services,01705,Wallace Sch - Mt Meigs Campus,Yes,No,No,No,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-18
1,ALABAMA,0100002,Alabama Youth Services,01706,McNeel Sch - Vacca Campus,Yes,No,No,No,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-18
2,ALABAMA,0100002,Alabama Youth Services,01876,Alabama Youth Services,No,No,No,No,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-18
3,ALABAMA,0100002,Alabama Youth Services,99995,AUTAUGA CAMPUS,Yes,No,No,No,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-18
4,ALABAMA,0100005,Albertville City,00870,Albertville Middle School,No,No,No,No,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-18


### Concatenate 2017-18 data to previous years

In [67]:
all_years = pd.concat([df_1718_final, prev_year_data], axis=0)

In [68]:
all_years['YEAR'].value_counts()

2011-12    101133
2017-18     97632
2015-16     96360
2013-14     95507
Name: YEAR, dtype: int64

In [69]:
full_table = all_years

In [70]:
full_table = full_table.astype(numeric_type_map)
# full_table.sort_index(inplace=True)
full_table.reset_index(inplace=True)
full_table['SCHID'] = full_table['SCHID'].astype(str).str.zfill(5)

Impute missing district state names from other years 

In [71]:
# Fill in LEA STATE with non-null values in LEAID group
full_table['LEA_STATE'] = full_table.groupby(['LEAID'])['LEA_STATE'].fillna(method='ffill').fillna(method='bfill')
full_table['LEA_STATE_NAME'] = full_table.groupby(['LEAID'])['LEA_STATE_NAME'].fillna(method='ffill').fillna(method='bfill')
full_table['LEA_NAME'] = full_table.groupby(['LEAID'])['LEA_NAME'].fillna(method='ffill').fillna(method='bfill')

In [72]:
full_table['SCH_NAME'] = full_table.groupby(['SCHID'])['SCH_NAME'].fillna(method='ffill').fillna(method='bfill')
full_table['SCH_ZIP '] = full_table.groupby(['SCHID'])['SCH_ZIP '].fillna(method='ffill').fillna(method='bfill')
full_table['SCH_CITY'] = full_table.groupby(['SCHID'])['SCH_CITY'].fillna(method='ffill').fillna(method='bfill')
full_table['SCH_ADDRESS'] = full_table.groupby(['SCHID'])['SCH_ADDRESS'].fillna(method='ffill').fillna(method='bfill')

Capitalize the first letter in each district name, school name, address.

In [73]:
full_table['LEA_NAME'] = full_table['LEA_NAME'].str.title()
full_table['SCH_NAME'] = full_table['SCH_NAME'].str.title()
full_table['SCH_ADDRESS'] = full_table['SCH_ADDRESS'].str.title()
full_table['SCH_CITY'] = full_table['SCH_CITY'].str.title()

Categorize each school as elementary, middle, high, or other using grade enrollment data.

In [74]:
full_table['SCH_LEVEL'] = full_table.apply(lambda row: categorize_school_level(row['SCH_GRADE_KG'], row['SCH_GRADE_G01'],
                            row['SCH_GRADE_G02'], row['SCH_GRADE_G03'], row['SCH_GRADE_G04'], row['SCH_GRADE_G05'],
                            row['SCH_GRADE_G06'], row['SCH_GRADE_G07'], row['SCH_GRADE_G08'], row['SCH_GRADE_G09'],
                            row['SCH_GRADE_G10'], row['SCH_GRADE_G11'], row['SCH_GRADE_G12']), axis=1
                            )

In [78]:
full_table.to_csv('/Users/cave/Desktop/pleiades_archive/discriminology/output/db_upload_schools.csv', index=False)

In [120]:
full_table.shape

(390632, 234)

In [ ]:
pg.df_to_rds()

### Group by districts and sum over fields

In [80]:
district = full_table.copy()

In [81]:
# district.columns = district.columns.droplevel(1)
district = district.astype(numeric_type_map)

In [82]:
district['TOTAL_SCHOOLS'] = 1.0
district['TOTAL_ENROLLMENT'] = district['TOT_ENR_F'] + district['TOT_ENR_M']

Custom aggregation dictionary - for string values, take the first non-null value.  For numeric fields, take the sum

In [83]:
agg_cols = ['LEA_NAME', 'LEA_STATE_NAME', 'TOTAL_SCHOOLS', 'TOTAL_ENROLLMENT',
'SCH_DISCWODIS_MULTOOS_HI_M',
 'SCH_DISCWDIS_ARR_IDEA_AM_M',
 'SCH_GTENR_IDEA_F',
 'SCH_DISCWODIS_SINGOOS_HP_F',
 'SCH_FTECOUNSELORS',
 'SCH_DISCWODIS_ARR_WH_M',
 'TOT_DISCWDIS_ARR_IDEA_M',
 'SCH_GTENR_TR_M',
 'SCH_DISCWODIS_MULTOOS_AS_M',
 'SCH_APENR_AM_F',
 'SCH_DISCWODIS_REF_BL_F',
 'SCH_DISCWODIS_ARR_AM_F',
 'TOT_DISCWODIS_REF_M',
 'SCH_IDEAENR_HI_M',
 'SCH_FTESERVICES_PSY',
 'SCH_DISCWDIS_MULTOOS_IDEA_BL_F',
 'TOT_APENR_F',
 'SCH_DISCWODIS_ARR_WH_F',
 'SCH_IDEAENR_HP_M',
 'SCH_DISCWDIS_MULTOOS_IDEA_TR_F',
 'SCH_DISCWDIS_REF_IDEA_WH_F',
 'SCH_DISCWODIS_MULTOOS_WH_M',
 'SCH_FTESECURITY_GUA',
 'SCH_DISCWODIS_SINGOOS_TR_M',
 'SCH_DISCWDIS_REF_IDEA_WH_M',
 'SCH_DISCWODIS_ARR_TR_M',
 'SCH_DISCWDIS_ARR_IDEA_BL_F',
 'SCH_DISCWDIS_ARR_IDEA_TR_M',
 'SCH_DISCWDIS_MULTOOS_IDEA_BL_M',
 'SCH_DISCWODIS_ARR_AM_M',
 'SCH_DISCWDIS_ARR_IDEA_AS_M',
 'SCH_DISCWDIS_REF_IDEA_BL_M',
 'SCH_GTENR_IDEA_M',
 'TOT_DISCWODIS_MULTOOS_M',
 'SCH_DISCWDIS_SINGOOS_IDEA_HI_F',
 'TOT_DISCWODIS_SINGOOS_F',
 'SCH_DISCWDIS_MULTOOS_IDEA_AM_F',
 'SCH_DISCWODIS_MULTOOS_TR_F',
 'TOT_DISCWODIS_SINGOOS_M',
 'SCH_ENR_HI_M',
 'SCH_APENR_HI_F',
 'SCH_IDEAENR_HI_F',
 'SCH_DISCWDIS_REF_IDEA_BL_F',
 'SCH_DISCWODIS_MULTOOS_AM_F',
 'TOT_DISCWDIS_MULTOOS_IDEA_M',
 'SCH_DISCWDIS_REF_IDEA_AS_M',
 'SCH_GTENR_HI_M',
 'SCH_DISCWODIS_REF_HI_M',
 'SCH_FTESERVICES_SOC',
 'SCH_DISCWDIS_SINGOOS_IDEA_AM_F',
 'SCH_DISCWODIS_REF_TR_M',
 'SCH_DISCWDIS_MULTOOS_IDEA_HP_M',
 'SCH_DISCWDIS_SINGOOS_IDEA_BL_M',
 'TOT_ENR_F',
 'SCH_GTENR_AS_F',
 'SCH_GTENR_HP_F',
 'SCH_ENR_BL_F',
 'SCH_DISCWODIS_SINGOOS_BL_M',
 'SCH_DISCWODIS_SINGOOS_TR_F',
 'SCH_DISCWDIS_MULTOOS_IDEA_WH_F',
 'SCH_DISCWDIS_MULTOOS_IDEA_AS_F',
 'SCH_DISCWODIS_SINGOOS_HP_M',
 'SCH_GTENR_WH_M',
 'SCH_DISCWODIS_REF_AM_F',
 'SCH_DISCWODIS_SINGOOS_AS_F',
 'SCH_DISCWDIS_MULTOOS_IDEA_HI_F',
 'SCH_DISCWDIS_SINGOOS_IDEA_HI_M',
 'SCH_DISCWODIS_SINGOOS_HI_M',
 'SCH_APENR_TR_M',
 'SCH_DISCWODIS_REF_HI_F',
 'SCH_DISCWODIS_ARR_BL_F',
 'SCH_ENR_TR_F',
 'SCH_DISCWODIS_SINGOOS_BL_F',
 'SCH_DISCWODIS_ARR_HP_F',
 'SCH_APENR_AS_M',
 'SCH_GTENR_LEP_M',
 'SCH_DISCWDIS_REF_IDEA_HI_F',
 'SCH_DISCWODIS_MULTOOS_BL_F',
 'SCH_DISCWODIS_REF_HP_M',
 'SCH_APENR_BL_F',
 'SCH_IDEAENR_AS_M',
 'TOT_DISCWODIS_ARR_M',
 'SCH_DISCWODIS_SINGOOS_HI_F',
 'SCH_DISCWDIS_SINGOOS_IDEA_WH_F',
 'SCH_IDEAENR_AM_F',
 'TOT_APENR_M',
 'SCH_DISCWDIS_SINGOOS_IDEA_TR_M',
 'SCH_DISCWODIS_REF_TR_F',
 'SCH_DISCWDIS_REF_IDEA_AM_M',
 'SCH_GTENR_BL_F',
 'SCH_IDEAENR_TR_F',
 'SCH_DISCWDIS_MULTOOS_IDEA_AM_M',
 'SCH_GTENR_HI_F',
 'SCH_DISCWODIS_ARR_AS_F',
 'SCH_DISCWDIS_SINGOOS_IDEA_TR_F',
 'TOT_DISCWODIS_MULTOOS_F',
 'SCH_DISCWODIS_REF_AS_M',
 'SCH_DISCWDIS_ARR_IDEA_WH_M',
 'SCH_DISCWDIS_SINGOOS_IDEA_BL_F',
 'TOT_DISCWDIS_SINGOOS_IDEA_M',
 'SCH_APENR_HI_M',
 'SCH_DISCWODIS_SINGOOS_WH_F',
 'SCH_DISCWDIS_ARR_IDEA_HP_M',
 'SCH_IDEAENR_AM_M',
 'SCH_APENR_AM_M',
 'SCH_GTENR_TR_F',
 'TOT_GTENR_F',
 'SCH_DISCWODIS_SINGOOS_AM_F',
 'SCH_APENR_WH_M',
 'SCH_DISCWODIS_REF_AM_M',
 'TOT_DISCWODIS_REF_F',
 'SCH_IDEAENR_BL_F',
 'SCH_DISCWODIS_SINGOOS_AS_M',
 'TOT_DISCWDIS_SINGOOS_IDEA_F',
 'SCH_DISCWODIS_ARR_HI_M',
 'SCH_DISCWDIS_ARR_IDEA_HP_F',
 'SCH_DISCWDIS_REF_IDEA_HI_M',
 'SCH_ENR_TR_M',
 'SCH_GTENR_BL_M',
 'SCH_DISCWDIS_SINGOOS_IDEA_AM_M',
 'SCH_DISCWODIS_ARR_HI_F',
 'SCH_IDEAENR_AS_F',
 'SCH_DISCWDIS_ARR_IDEA_HI_F',
 'SCH_APENR_HP_F',
 'TOT_IDEAENR_M',
 'SCH_GTENR_AM_M',
 'SCH_ENR_BL_M',
 'SCH_DISCWDIS_MULTOOS_IDEA_AS_M',
 'SCH_ENR_AM_M',
 'SCH_DISCWODIS_MULTOOS_WH_F',
 'SCH_ENR_HP_M',
 'SCH_IDEAENR_BL_M',
 'SCH_IDEAENR_TR_M',
 'SCH_APENR_WH_F',
 'SCH_DISCWDIS_SINGOOS_IDEA_AS_F',
 'SCH_ENR_AM_F',
 'SCH_DISCWDIS_REF_IDEA_TR_M',
 'SCH_DISCWODIS_REF_AS_F',
 'SCH_DISCWDIS_ARR_IDEA_AM_F',
 'SCH_ENR_WH_M',
 'SCH_DISCWODIS_REF_BL_M',
 'SCH_DISCWODIS_MULTOOS_AM_M',
 'SCH_DISCWDIS_ARR_IDEA_HI_M',
 'SCH_DISCWODIS_REF_HP_F',
 'SCH_DISCWDIS_SINGOOS_IDEA_AS_M',
 'SCH_DISCWODIS_ARR_AS_M',
 'SCH_IDEAENR_WH_M',
 'SCH_DISCWDIS_MULTOOS_IDEA_HP_F',
 'TOT_DISCWDIS_REF_IDEA_M',
 'SCH_FTESECURITY_LEO',
 'SCH_DISCWODIS_MULTOOS_TR_M',
 'SCH_IDEAENR_HP_F',
 'SCH_DISCWODIS_MULTOOS_HI_F',
 'SCH_DISCWDIS_ARR_IDEA_WH_F',
 'SCH_GTENR_HP_M',
 'SCH_DISCWDIS_MULTOOS_IDEA_TR_M',
 'SCH_DISCWDIS_ARR_IDEA_AS_F',
 'SCH_DISCWODIS_REF_WH_M',
 'SCH_DISCWODIS_ARR_BL_M',
 'TOT_IDEAENR_F',
 'TOT_DISCWDIS_ARR_IDEA_F',
 'SCH_DISCWODIS_MULTOOS_AS_F',
 'SCH_DISCWDIS_MULTOOS_IDEA_WH_M',
 'SCH_APENR_TR_F',
 'SCH_ENR_WH_F',
 'SCH_DISCWODIS_REF_WH_F',
 'SCH_APENR_HP_M',
 'SCH_DISCWODIS_MULTOOS_BL_M',
 'SCH_GTENR_AM_F',
 'TOT_DISCWDIS_MULTOOS_IDEA_F',
 'SCH_DISCWDIS_REF_IDEA_HP_M',
 'SCH_ENR_HI_F',
 'SCH_DISCWDIS_REF_IDEA_HP_F',
 'TOT_ENR_M',
 'SCH_ENR_AS_F',
 'SCH_DISCWDIS_REF_IDEA_AM_F',
 'SCH_DISCWODIS_ARR_HP_M',
 'SCH_APENR_AS_F',
 'TOT_DISCWODIS_ARR_F',
 'SCH_ENR_AS_M',
 'SCH_GTENR_WH_F',
 'SCH_DISCWDIS_SINGOOS_IDEA_HP_F',
 'SCH_GTENR_LEP_F',
 'SCH_IDEAENR_WH_F',
 'SCH_DISCWODIS_SINGOOS_AM_M',
 'SCH_DISCWODIS_SINGOOS_WH_M',
 'SCH_DISCWDIS_ARR_IDEA_TR_F',
 'SCH_ENR_HP_F',
 'SCH_DISCWODIS_MULTOOS_HP_M',
 'SCH_DISCWDIS_REF_IDEA_TR_F',
 'SCH_GTENR_AS_M',
 'SCH_DISCWDIS_SINGOOS_IDEA_WH_M',
 'TOT_GTENR_M',
 'SCH_DISCWODIS_ARR_TR_F',
 'SCH_DISCWDIS_SINGOOS_IDEA_HP_M',
 'SCH_DISCWODIS_MULTOOS_HP_F',
 'SCH_DISCWDIS_ARR_IDEA_BL_M',
 'SCH_DISCWDIS_REF_IDEA_AS_F',
 'SCH_APENR_BL_M',
 'TOT_DISCWDIS_REF_IDEA_F',
 'SCH_DISCWDIS_MULTOOS_IDEA_HI_M']

In [84]:
agg_dict = {k:sum if k not in {'LEA_NAME', 'LEA_STATE_NAME'} else 'first' for k in agg_cols}

In [86]:
assert(len(agg_dict) == len(agg_cols))

In [87]:
grouped_by_distyr = district.groupby(['LEAID', 'YEAR'])[agg_cols].agg(agg_dict)
grouped_by_distyr = grouped_by_distyr[agg_cols]
grouped_by_distyr.reset_index(inplace=True)
grouped_by_distyr.sort_values(by=['LEAID', 'YEAR'], inplace=True)

Read geodata from file and join to district aggregation.

In [88]:
sch_geo_1516 = pd.read_excel('/Users/cave/Desktop/pleiades_archive/discriminology/Geocoded Schools:Districts/2015:16/Schools_EDGE_GEOCODE_PUBLICSCH_1516.xlsx', dtype={'NCESSCH': str})
dist_geo_1516 = pd.read_excel('/Users/cave/Desktop/pleiades_archive/discriminology/Geocoded Schools:Districts/2015:16/Districts_EDGE_GEOCODE_PUBLICLEA_1516.xlsx', dtype={'LEAID': str})


In [89]:
sch_geo_1516['NCESSCH'] = sch_geo_1516['NCESSCH'].str.zfill(12)
dist_geo_1516['LEAID'] = dist_geo_1516['LEAID'].str.zfill(7)
sch_geo_1516.NCESSCH.nunique()

102209

In [90]:
sch_geo_1819 = pd.read_excel('/Users/cave/Desktop/pleiades_archive/discriminology/Geocoded Schools:Districts/2018:19/Schools_EDGE_GEOCODE_PUBLICSCH_1819.xlsx', dtype={'NCESSCH': str})
dist_geo_1819 = pd.read_excel('/Users/cave/Desktop/pleiades_archive/discriminology/Geocoded Schools:Districts/2018:19/Districts_EDGE_GEOCODE_PUBLICLEA_1819.xlsx', dtype={'LEAID': str})

In [91]:
sch_geo_1819['NCESSCH'] = sch_geo_1819['NCESSCH'].str.zfill(12)
dist_geo_1819['LEAID'] = dist_geo_1819['LEAID'].str.zfill(7)
sch_geo_1819.NCESSCH.nunique()

102176

In [92]:
geo_data = pd.read_excel('/Users/cave/Desktop/pleiades_archive/discriminology/LEA Profile Info.xlsx', dtype={'LEAID': str})
geo_data['LEAID'] = geo_data['LEAID'].str.zfill(7)

In [93]:
len(geo_data)

16758

In [94]:
len(set(grouped_by_distyr['LEAID'].values))

31008

In [95]:
len(set(geo_data['LEAID'].values).intersection(set(grouped_by_distyr['LEAID'].values)))

16333

In [96]:
district_geo = geo_data[['LEAID', 'LEA_ADDRESS', 'LEA_CITY', 'LEA_ZIP',
       'CJJ', 'LEA_ENR', 'LEA_SCHOOLS', 'Latitude', 'Longitude']]

In [97]:
district_geo.head()

,LEAID,LEA_ADDRESS,LEA_CITY,LEA_ZIP,CJJ,LEA_ENR,LEA_SCHOOLS,Latitude,Longitude
0,0100002,1000 INDUSTRIAL SCHOOL ROAD,MT. MEIGS,36057,Yes,3674,3,32.371901,-86.083791
1,0100005,107 WEST MAIN STREET,ALBERTVILLE,35950,No,4712,6,34.268415,-86.209216
2,0100006,12380 US HIGHWAY 431 S,GUNTERSVILLE,35976,No,5624,14,34.305366,-86.287334
3,0100007,2810 METROPOLITAN WAY,HOOVER,35243,No,14054,16,33.445932,-86.750203
4,0100008,211 CELTIC DRIVE,MADISON,35758,Yes,9206,11,34.686749,-86.745762


In [98]:
district_group_w_geo = pd.merge(grouped_by_distyr, district_geo, left_on='LEAID', right_on='LEAID', how='left')

In [99]:
district_group_w_geo.head()

,LEAID,YEAR,LEA_NAME,LEA_STATE_NAME,TOTAL_SCHOOLS,TOTAL_ENROLLMENT,SCH_DISCWODIS_MULTOOS_HI_M,SCH_DISCWDIS_ARR_IDEA_AM_M,SCH_GTENR_IDEA_F,SCH_DISCWODIS_SINGOOS_HP_F,...,TOT_DISCWDIS_REF_IDEA_F,SCH_DISCWDIS_MULTOOS_IDEA_HI_M,LEA_ADDRESS,LEA_CITY,LEA_ZIP,CJJ,LEA_ENR,LEA_SCHOOLS,Latitude,Longitude
0,100002,2011-12,Alabama Youth Services,ALABAMA,4.0,1098.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100002,2013-14,Alabama Youth Services,ALABAMA,3.0,3702.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100002,2015-16,Alabama Youth Services,ALABAMA,4.0,1126.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100005,2011-12,Albertville City,ALABAMA,6.0,4149.0,8.0,0.0,0.0,0.0,...,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100005,2013-14,Albertville City,ALABAMA,6.0,4723.0,9.0,0.0,0.0,0.0,...,0.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [100]:
text_cols = ['LEAID', 'LEA_NAME', "YEAR", "LEA_STATE", "LEA_STATE_NAME",
    "LEAID", "LEA_NAME", 'LEA_ADDRESS', 'LEA_CITY', 'CJJ',
    ]

In [101]:
district_create = [print(f'"{k}" double precision,') if k not in text_cols else print(f'"{k}" text,') for k in district_group_w_geo.columns]

"LEAID" text,
"YEAR" text,
"LEA_NAME" text,
"LEA_STATE_NAME" text,
"TOTAL_SCHOOLS" double precision,
"TOTAL_ENROLLMENT" double precision,
"SCH_DISCWODIS_MULTOOS_HI_M" double precision,
"SCH_DISCWDIS_ARR_IDEA_AM_M" double precision,
"SCH_GTENR_IDEA_F" double precision,
"SCH_DISCWODIS_SINGOOS_HP_F" double precision,
"SCH_FTECOUNSELORS" double precision,
"SCH_DISCWODIS_ARR_WH_M" double precision,
"TOT_DISCWDIS_ARR_IDEA_M" double precision,
"SCH_GTENR_TR_M" double precision,
"SCH_DISCWODIS_MULTOOS_AS_M" double precision,
"SCH_APENR_AM_F" double precision,
"SCH_DISCWODIS_REF_BL_F" double precision,
"SCH_DISCWODIS_ARR_AM_F" double precision,
"TOT_DISCWODIS_REF_M" double precision,
"SCH_IDEAENR_HI_M" double precision,
"SCH_FTESERVICES_PSY" double precision,
"SCH_DISCWDIS_MULTOOS_IDEA_BL_F" double precision,
"TOT_APENR_F" double precision,
"SCH_DISCWODIS_ARR_WH_F" double precision,
"SCH_IDEAENR_HP_M" double precision,
"SCH_DISCWDIS_MULTOOS_IDEA_TR_F" double precision,
"SCH_DISCWDIS_REF_IDEA_WH

In [102]:
dist_type_map = {k:v for k,v in type_map.items() if k in district_group_w_geo.columns}

In [103]:
district_group_w_geo = district_group_w_geo.astype(dist_type_map)

In [108]:
district_group_w_geo.to_csv('/Users/cave/Desktop/pleiades_archive/discriminology/output/db_upload_districts.csv', index=False)

### Code Sandbox - everything below only needs to be run once

In [ ]:
# descriptions = pd.concat(desc_frames_1112, axis=0)
# descriptions.drop_duplicates(inplace=True)
# descriptions.set_index('Field Name', inplace=True)
# descriptions = descriptions.loc[fields_1112]
# descriptions.columns = ['2011_12_description']
# descriptions.reset_index().to_csv('/Users/cave/Desktop/2011_12_field_descriptions.csv')

# descriptions = pd.concat(desc_frames_1314, axis=0)
# descriptions.drop_duplicates(inplace=True)
# descriptions.set_index('Field Name', inplace=True)
# descriptions = descriptions.loc[fields_1314]
# descriptions.columns = ['2013_14_description']
# descriptions.reset_index().to_csv('/Users/cave/Desktop/2013_14_field_descriptions.csv')